In [2]:
players = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("../datasets/player.csv")

In [5]:
players.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- birthday: string (nullable = true)
 |-- height: string (nullable = true)
 |-- weight: string (nullable = true)



In [8]:
players.show(10)

+---+-------------+------------------+------------------+-------------------+------+------+
| id|player_api_id|       player_name|player_fifa_api_id|           birthday|height|weight|
+---+-------------+------------------+------------------+-------------------+------+------+
|  1|       505942|Aaron Appindangoye|            218353|1992-02-29 00:00:00|182.88|   187|
|  2|       155782|   Aaron Cresswell|            189615|1989-12-15 00:00:00|170.18|   146|
|  3|       162549|       Aaron Doran|            186170|1991-05-13 00:00:00|170.18|   163|
|  4|        30572|     Aaron Galindo|            140161|1982-05-08 00:00:00|182.88|   198|
|  5|        23780|      Aaron Hughes|             17725|1979-11-08 00:00:00|182.88|   154|
|  6|        27316|        Aaron Hunt|            158138|1986-09-04 00:00:00|182.88|   161|
|  7|       564793|        Aaron Kuhl|            221280|1996-01-30 00:00:00|172.72|   146|
|  8|        30895|      Aaron Lennon|            152747|1987-04-16 00:00:00| 16

In [9]:
player_attributes = spark.read\
            .format("csv")\
            .option("header","true")\
            .load("../datasets/player_attributes.csv")

In [10]:
player_attributes.printSchema()

root
 |-- id: string (nullable = true)
 |-- player_fifa_api_id: string (nullable = true)
 |-- player_api_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- potential: string (nullable = true)
 |-- preferred_foot: string (nullable = true)
 |-- attacking_work_rate: string (nullable = true)
 |-- defensive_work_rate: string (nullable = true)
 |-- crossing: string (nullable = true)
 |-- finishing: string (nullable = true)
 |-- heading_accuracy: string (nullable = true)
 |-- short_passing: string (nullable = true)
 |-- volleys: string (nullable = true)
 |-- dribbling: string (nullable = true)
 |-- curve: string (nullable = true)
 |-- free_kick_accuracy: string (nullable = true)
 |-- long_passing: string (nullable = true)
 |-- ball_control: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- sprint_speed: string (nullable = true)
 |-- agility: string (nullable = true)
 |-- reactions: string (nullable = true

In [11]:
player_attributes.show(5)

+---+------------------+-------------+-------------------+--------------+---------+--------------+-------------------+-------------------+--------+---------+----------------+-------------+-------+---------+-----+------------------+------------+------------+------------+------------+-------+---------+-------+----------+-------+-------+--------+----------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+--------------+-----------+
| id|player_fifa_api_id|player_api_id|               date|overall_rating|potential|preferred_foot|attacking_work_rate|defensive_work_rate|crossing|finishing|heading_accuracy|short_passing|volleys|dribbling|curve|free_kick_accuracy|long_passing|ball_control|acceleration|sprint_speed|agility|reactions|balance|shot_power|jumping|stamina|strength|long_shots|aggression|interceptions|positioning|vision|penalties|marking|standing_tackle|sliding_tackle|gk_diving|gk_handling|gk_kicking|gk_posit

In [13]:
players.count() , player_attributes.count()

(11060, 183978)

In [14]:
player_attributes.select("player_api_id")\
                    .distinct()\
                    .count()

11060

In [16]:
players = players.drop("id","player_fifa_api_id")
players.columns

['player_api_id', 'player_name', 'birthday', 'height', 'weight']

In [17]:
player_attributes.columns

['id',
 'player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [18]:
player_attributes = player_attributes.drop(
        'id',
         'player_fifa_api_id',
        'preferred_foot',
         'attacking_work_rate',
         'defensive_work_rate',
         'crossing',
        'jumping',
        'sprint_speed',
        'balance',
        'aggression',
        'short_passing',
        'potential'
)
player_attributes.columns

['player_api_id',
 'date',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

In [19]:
player_attributes = player_attributes.dropna()
players = players.dropna()

In [20]:
players.count() , player_attributes.count()

(11060, 181265)

In [21]:
from pyspark.sql.functions import udf

In [24]:
year_extract_udf = udf(lambda date:date.split("-")[0])

In [25]:
player_attributes = player_attributes.withColumn(
            "year",
            year_extract_udf(player_attributes.date)
)
player_attributes.show(5)

+-------------+-------------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+--------------+-----------+----+
|player_api_id|               date|overall_rating|finishing|heading_accuracy|volleys|dribbling|curve|free_kick_accuracy|long_passing|ball_control|acceleration|agility|reactions|shot_power|stamina|strength|long_shots|interceptions|positioning|vision|penalties|marking|standing_tackle|sliding_tackle|gk_diving|gk_handling|gk_kicking|gk_positioning|gk_reflexes|year|
+-------------+-------------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+-----

In [26]:
player_attributes = player_attributes.drop("date")
player_attributes.printSchema()

root
 |-- player_api_id: string (nullable = true)
 |-- overall_rating: string (nullable = true)
 |-- finishing: string (nullable = true)
 |-- heading_accuracy: string (nullable = true)
 |-- volleys: string (nullable = true)
 |-- dribbling: string (nullable = true)
 |-- curve: string (nullable = true)
 |-- free_kick_accuracy: string (nullable = true)
 |-- long_passing: string (nullable = true)
 |-- ball_control: string (nullable = true)
 |-- acceleration: string (nullable = true)
 |-- agility: string (nullable = true)
 |-- reactions: string (nullable = true)
 |-- shot_power: string (nullable = true)
 |-- stamina: string (nullable = true)
 |-- strength: string (nullable = true)
 |-- long_shots: string (nullable = true)
 |-- interceptions: string (nullable = true)
 |-- positioning: string (nullable = true)
 |-- vision: string (nullable = true)
 |-- penalties: string (nullable = true)
 |-- marking: string (nullable = true)
 |-- standing_tackle: string (nullable = true)
 |-- sliding_tackle:

In [27]:
pa_2016 = player_attributes.filter(player_attributes["year"] == "2016")

In [32]:
pa_2016.select(pa_2016.player_api_id)\
        .distinct()\
        .count()

5586

In [41]:
pa_striker_2016 = pa_2016.groupBy(pa_2016.player_api_id)\
                        .agg(
                            {"finishing" : "avg",\
                            "shot_power" : "avg",\
                            "acceleration" : "avg"},\
                        )\
                        .withColumnRenamed("avg(finishing)","finishing")\
                        .withColumnRenamed("avg(shot_power)","shot_power")\
                        .withColumnRenamed("avg(acceleration)","acceleration")
pa_striker_2016.show(5)

+-------------+-----------------+-----------------+----------+
|player_api_id|        finishing|     acceleration|shot_power|
+-------------+-----------------+-----------------+----------+
|       309726|75.44444444444444|74.11111111111111|      76.0|
|        26112|             53.0|             51.0|      76.0|
|        38433|            68.25|             74.0|      74.0|
|       295060|             25.0|             62.0|      40.0|
|       161396|             29.0|             72.0|      69.0|
+-------------+-----------------+-----------------+----------+
only showing top 5 rows



In [42]:
weight_finishing = 1
weight_shot_power = 2
weight_acceleration = 1
total_weight = weight_finishing + weight_shot_power + weight_acceleration

In [45]:
strikers = pa_striker_2016.withColumn("striker_grade",
                                     (pa_striker_2016.finishing * weight_finishing + \
                                      pa_striker_2016.acceleration * weight_acceleration + \
                                      pa_striker_2016.shot_power * weight_shot_power \
                                     )/total_weight)

In [47]:
strikers.count()

5586

In [49]:
strikers.orderBy(strikers.striker_grade.desc()).show(10)

+-------------+-----------------+------------+----------+-----------------+
|player_api_id|        finishing|acceleration|shot_power|    striker_grade|
+-------------+-----------------+------------+----------+-----------------+
|        20276|             85.0|        84.0|      94.0|            89.25|
|        37412|             90.0|        92.0|      87.0|             89.0|
|        38817|             88.0|        90.0|      88.5|            88.75|
|        32118|             85.0|        82.0|      93.0|            88.25|
|        31921|             81.0|        93.0|      87.0|             87.0|
|        30834|             85.0|        90.0|      86.0|            86.75|
|       303824|73.42857142857143|        91.0|      88.0|85.10714285714286|
|       129944|             83.0|        89.0|      84.0|             85.0|
|       158263|             77.0|        90.0|      86.0|            84.75|
|       150565|             88.0|        95.0|      78.0|            84.75|
+-----------

In [50]:
strikers=strikers.drop("finishing","","acceleration","shot_power")

In [51]:
strikers.orderBy(strikers.striker_grade.desc()).show(10)

+-------------+-----------------+
|player_api_id|    striker_grade|
+-------------+-----------------+
|        20276|            89.25|
|        37412|             89.0|
|        38817|            88.75|
|        32118|            88.25|
|        31921|             87.0|
|        30834|            86.75|
|       303824|85.10714285714286|
|       129944|             85.0|
|       150565|            84.75|
|       158263|            84.75|
+-------------+-----------------+
only showing top 10 rows



In [52]:
strikers = strikers.filter(strikers.striker_grade > 70)\
                    .sort(strikers.striker_grade.desc())
strikers.show(10)

+-------------+-----------------+
|player_api_id|    striker_grade|
+-------------+-----------------+
|        20276|            89.25|
|        37412|             89.0|
|        38817|            88.75|
|        32118|            88.25|
|        31921|             87.0|
|        30834|            86.75|
|       303824|85.10714285714286|
|       129944|             85.0|
|       150565|            84.75|
|       158263|            84.75|
+-------------+-----------------+
only showing top 10 rows



In [53]:
strikers.count(),players.count()

(1609, 11060)

In [54]:
players.show(1)

+-------------+------------------+-------------------+------+------+
|player_api_id|       player_name|           birthday|height|weight|
+-------------+------------------+-------------------+------+------+
|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
+-------------+------------------+-------------------+------+------+
only showing top 1 row



In [55]:
striker_details = players.join(strikers,players.player_api_id == strikers.player_api_id)
striker_details.columns

['player_api_id',
 'player_name',
 'birthday',
 'height',
 'weight',
 'player_api_id',
 'striker_grade']

In [56]:
striker_details.count()

1609

In [57]:
striker_details = players.join(strikers,["player_api_id"])
striker_details.columns

['player_api_id',
 'player_name',
 'birthday',
 'height',
 'weight',
 'striker_grade']

In [58]:
striker_details.count()

1609

In [60]:
striker_details.orderBy(striker_details.striker_grade.desc()).show(5)

+-------------+--------------+-------------------+------+------+-------------+
|player_api_id|   player_name|           birthday|height|weight|striker_grade|
+-------------+--------------+-------------------+------+------+-------------+
|        20276|          Hulk|1986-07-25 00:00:00|180.34|   187|        89.25|
|        37412| Sergio Aguero|1988-06-02 00:00:00|172.72|   163|         89.0|
|        38817|  Carlos Tevez|1984-02-05 00:00:00|172.72|   157|        88.75|
|        32118|Lukas Podolski|1985-06-04 00:00:00|182.88|   183|        88.25|
|        31921|   Gareth Bale|1989-07-16 00:00:00|182.88|   163|         87.0|
+-------------+--------------+-------------------+------+------+-------------+
only showing top 5 rows



In [61]:
from pyspark.sql.functions import broadcast

In [62]:
striker_details = players.select("player_api_id","player_name")\
                        .join(
                            broadcast(strikers),
                                ["Player_api_id"],
                                "inner"
                            )

In [63]:
striker_details.count()

1609

In [65]:
striker_details.sort(striker_details.striker_grade.desc()).show(5)

+-------------+--------------+-------------+
|player_api_id|   player_name|striker_grade|
+-------------+--------------+-------------+
|        20276|          Hulk|        89.25|
|        37412| Sergio Aguero|         89.0|
|        38817|  Carlos Tevez|        88.75|
|        32118|Lukas Podolski|        88.25|
|        31921|   Gareth Bale|         87.0|
+-------------+--------------+-------------+
only showing top 5 rows



In [66]:
players.count(), player_attributes.count()

(11060, 181265)

In [67]:
players.show()

+-------------+--------------------+-------------------+------+------+
|player_api_id|         player_name|           birthday|height|weight|
+-------------+--------------------+-------------------+------+------+
|       505942|  Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       155782|     Aaron Cresswell|1989-12-15 00:00:00|170.18|   146|
|       162549|         Aaron Doran|1991-05-13 00:00:00|170.18|   163|
|        30572|       Aaron Galindo|1982-05-08 00:00:00|182.88|   198|
|        23780|        Aaron Hughes|1979-11-08 00:00:00|182.88|   154|
|        27316|          Aaron Hunt|1986-09-04 00:00:00|182.88|   161|
|       564793|          Aaron Kuhl|1996-01-30 00:00:00|172.72|   146|
|        30895|        Aaron Lennon|1987-04-16 00:00:00| 165.1|   139|
|       528212|        Aaron Lennox|1993-02-19 00:00:00| 190.5|   181|
|       101042|       Aaron Meijers|1987-10-28 00:00:00|175.26|   170|
|        23889|       Aaron Mokoena|1980-11-25 00:00:00|182.88|   181|
|     

In [68]:
player_attributes.show(2)

+-------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+--------------+-----------+----+
|player_api_id|overall_rating|finishing|heading_accuracy|volleys|dribbling|curve|free_kick_accuracy|long_passing|ball_control|acceleration|agility|reactions|shot_power|stamina|strength|long_shots|interceptions|positioning|vision|penalties|marking|standing_tackle|sliding_tackle|gk_diving|gk_handling|gk_kicking|gk_positioning|gk_reflexes|year|
+-------------+--------------+---------+----------------+-------+---------+-----+------------------+------------+------------+------------+-------+---------+----------+-------+--------+----------+-------------+-----------+------+---------+-------+---------------+--------------+---------+-----------+----------+-

In [69]:
players_heading_acc = player_attributes.select("player_api_id","heading_accuracy")\
                                        .join(
                                        broadcast(players),
                                        ["player_api_id"],
                                        "inner"
                                        )

In [70]:
players_heading_acc.columns

['player_api_id',
 'heading_accuracy',
 'player_name',
 'birthday',
 'height',
 'weight']

In [71]:
players_heading_acc = player_attributes.select("player_api_id","heading_accuracy")\
                                        .join(
                                        broadcast(players),
                                        players.player_api_id == player_attributes.player_api_id
                                        )

In [72]:
players_heading_acc.columns

['player_api_id',
 'heading_accuracy',
 'player_api_id',
 'player_name',
 'birthday',
 'height',
 'weight']

In [73]:
players_heading_acc.count()

181265

In [74]:
short_count = spark.sparkContext.accumulator(0)
medium_low_count = spark.sparkContext.accumulator(0)
medium_high_count = spark.sparkContext.accumulator(0)
tall_count = spark.sparkContext.accumulator(0)

In [78]:
def count_players_by_heights(row):
    height = float(row.height)
    if(height <= 175):
        short_count.add(1)
    elif(height > 175 and height<=183):
        medium_low_count.add(1)
    elif(height > 183 and height<=195):
        medium_high_count.add(1)
    elif(height>195):
        tall_count.add(1)

In [79]:
players_heading_acc.show(10)

+-------------+----------------+-------------+------------------+-------------------+------+------+
|player_api_id|heading_accuracy|player_api_id|       player_name|           birthday|height|weight|
+-------------+----------------+-------------+------------------+-------------------+------+------+
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              71|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              70|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       505942|              70|       505942|Aaron Appindangoye|1992-02-29 00:00:00|182.88|   187|
|       155782|              58|       155782|   Aaron Cresswell|1989-12-15 00:00:00|170.18|   146|
|       155782|              58|       155782|   Aaron Cresswell|1989-12-15 00:00:00|170.18|   146|


In [80]:
players_heading_acc.foreach(lambda x:count_players_by_heights(x))

In [82]:
all_players = [short_count.value,
               medium_low_count.value,
               medium_high_count.value,
               tall_count.value]

all_players

[18977, 97399, 61518, 3371]

In [89]:
short_ha_count = spark.sparkContext.accumulator(0)
medium_ha_low_count = spark.sparkContext.accumulator(0)
medium_ha_high_count = spark.sparkContext.accumulator(0)
tall_ha_count = spark.sparkContext.accumulator(0)

In [90]:
def count_players_by_height_and_heading_accuracy(row,threshold_score):
    height = float(row.height)
    ha = float(row.heading_accuracy)
    
    if(ha <= threshold_score):
        return
    
    if(height <= 175):
        short_ha_count.add(1)
    elif(height > 175 and height<=183):
        medium_ha_low_count.add(1)
    elif(height > 183 and height<=195):
        medium_ha_high_count.add(1)
    elif(height>195):
        tall_ha_count.add(1)

In [91]:
players_heading_acc.foreach(lambda x:count_players_by_height_and_heading_accuracy(x,60))

In [92]:
all_players_above_threshold = [short_ha_count.value,
               medium_ha_low_count.value,
               medium_ha_high_count.value,
               tall_ha_count.value]

all_players_above_threshold

[3653, 41448, 40270, 1573]

In [93]:
percentage_values = [short_ha_count.value / short_count.value * 100,
                     medium_ha_low_count.value / medium_low_count.value * 100,
                     medium_ha_high_count.value / medium_high_count.value * 100,
                     tall_ha_count.value / tall_count.value * 100
                    ]
percentage_values

[19.249617958581442, 42.55485169252251, 65.46051562144413, 46.66271136161376]

In [94]:
pa_2016.columns

['player_api_id',
 'overall_rating',
 'finishing',
 'heading_accuracy',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'agility',
 'reactions',
 'shot_power',
 'stamina',
 'strength',
 'long_shots',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes',
 'year']

In [95]:
pa_2016.select("player_api_id","overall_rating")\
        .coalesce(1)\
        .write\
        .option("header","true")\
        .csv("players_overall.csv")

In [96]:
pa_2016.select("player_api_id","overall_rating")\
        .write\
        .json("players_overall.json")